In [3]:
# 1) Gerekli kütüphaneler - Colab üzerinde çalıştırmak için
!pip install -q sentence-transformers faiss-cpu transformers[torch] accelerate
!pip install -q gradio==4.44.0 websockets>=13.0.0


In [4]:
# 2) Kütüphaneleri import et
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import numpy as np
import pickle
import os


# Gradio
import gradio as gr

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or
/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:82: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [5]:
# ## 3) Küçük Finans Metin Koleksiyonu

finans_docs = [
{
"title": "Enflasyon Nedir",
"text": (
"Enflasyon, bir ekonomide genel fiyat düzeyinin zaman içinde artmasıdır. "
"Enflasyon, satın alma gücünü azaltır ve merkez bankalarının para politikası kararlarında belirleyici olur."
),
},
{
"title": "Faiz Oranları ve Tahvil",
"text": (
"Faiz oranları, borç verenlerin borç verdikleri paranın kullanım karşılığı aldığı ücrettir. "
"Tahviller sabit gelirli menkul kıymetlerdir ve genellikle sabit faiz ödemesi yaparlar."
),
},
{
"title": "Döviz Kurları",
"text": (
"Döviz kuru iki para birimi arasındaki dönüşüm oranıdır. Kurlar; merkez bankası politikaları, cari açık, faiz farkları ve piyasa beklentileriyle etkilenir."
),
},
{
"title": "Hisse Senetleri ve Piyasa",
"text": (
"Hisse senetleri bir şirkete ortaklık hakkı sağlar. Piyasa değeri, arz-talep ve yatırımcı beklentileri tarafından belirlenir."
),
},
{
"title": "Kripto Paralar ve Blockchain",
"text": (
"Kripto paralar dijital varlıklardır; blockchain teknolojisi ile dağıtık defter mantığı kullanılır. Volatilite yüksektir ve regülasyonlar önem taşır."
),
},
{
"title": "Yatırım Çeşitleri",
"text": (
"Yatırımlar; hisse, tahvil, emlak, yatırım fonları, vadeli işlemler ve kripto gibi çeşitli araçlara dağıtılabilir. Diversifikasyon risk yönetiminde kullanılır."
),
},
]

In [7]:
# ## 4) Embedding Oluşturma ve FAISS Index
# sentence-transformers/all-MiniLM-L6-v2 modeli hızlı ve hafiftir; Colab üzerinde iyi çalışır.


# %%
embed_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(embed_model_name)


# Dokümanları parçalayıp embedding oluştur
texts = [d["text"] for d in finans_docs]
meta = [{"title": d["title"]} for d in finans_docs]


embeddings = embed_model.encode(texts, convert_to_numpy=True, show_progress_bar=True)


# FAISS index oluştur (L2) - normalize edip inner product da kullanılabilir ama L2 ile gideceğiz
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)
print(f"FAISS index: {index.ntotal} vektör eklendi. Dimension = {dimension}")


# İsterseniz index'i kaydedin
with open('meta.pkl', 'wb') as f:
  pickle.dump(meta, f)


faiss.write_index(index, 'finans_faiss.index')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

FAISS index: 6 vektör eklendi. Dimension = 384


In [9]:
# ## 5) Retriever + Generator (Basit RAG Mantığı)
# - Soruyu embed edip FAISS'ten en yakın dokümanı alacağız
# - Doküman + soruyu birleşik prompt ile Flan-T5'e verip cevap oluşturacağız


# %%
# Tokenizer ve model (Flan-T5-base)
model_name = "google/flan-t5-base"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


gen_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer, device=0 if __import__('torch').cuda.is_available() else -1)


# %%
# Basit retriever fonksiyonu
def retrieve(query, k=2):
  q_emb = embed_model.encode([query], convert_to_numpy=True)
  D, I = index.search(q_emb, k)
  results = []
  for idx in I[0]:
    results.append({"text": texts[idx], "meta": meta[idx]})
  return results


# Prompt hazırlama
def make_prompt(question, contexts):
  prompt = "Aşağıdaki bağlamları kullanarak Türkçe olarak açık ve kısa bir cevap ver. Eğer bilgi bağlamlarda yoksa, bilgiyi bildiğini iddia etme, bilmiyorum de.\n\n"
  for i, c in enumerate(contexts):
    prompt += f"Bağlam {i+1}: {c['text']}\n"
    prompt += f"\nSoru: {question}\nCevap:"
  return prompt


# RAG answer fonksiyonu
def rag_answer(question):
  contexts = retrieve(question, k=2)
  prompt = make_prompt(question, contexts)
  out = gen_pipeline(prompt, max_length=256, do_sample=False)[0]['generated_text']
  return out

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu


In [12]:
# ## 6) Gradio Arayüzü


# %%
# Gradio arayüzü kur
import gradio as gr

def respond(q):
    if not q or q.strip() == "":
        return "Lütfen bir soru girin."
    return rag_answer(q)

with gr.Blocks() as demo:
    gr.Markdown("# Finans RAG Chatbot\nFinans konularında (yatırım, enflasyon, faiz, döviz, kripto vb.) soru sorabilirsiniz.")
    with gr.Row():
        inp = gr.Textbox(lines=2, placeholder="Soru yazın...", label="Soru")
        out = gr.Textbox(label="Cevap")
    btn = gr.Button("Gönder")
    btn.click(respond, inputs=inp, outputs=out)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


/usr/local/lib/python3.12/dist-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://9c63a6ef8548b55e9a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [14]:
# ## 7) Kaydetme ve GitHub için dosyalar
# Aşağıdaki içerikleri `requirements.txt` ve `README.md` olarak kaydedin (kod takip eden hücrede yapılmıştır).


# %%
requirements = '''
sentence-transformers
faiss-cpu
transformers[torch]
gradio==3.39.1
accelerate
'''
with open('requirements.txt', 'w') as f:
  f.write(requirements)


readme = '''# Finans RAG Chatbot


Bu proje, açık kaynak modeller kullanılarak hazırlanmış bir RAG (Retrieval-Augmented Generation) chatbottur.


## Amaç
Finans konularında (yatırım, enflasyon, faiz, döviz, kripto vb.) kullanıcı sorularını, küçük bir doküman koleksiyonundan yararlanarak cevaplamak.


## Veri Seti
Proje örnek amaçlı küçük bir metin koleksiyonu kullanır. İstendiğinde daha büyük bir dataset (HuggingFace, web scraping vb.) ile genişletilebilir.


## Kullanılan Yöntemler
- Embedding: sentence-transformers/all-MiniLM-L6-v2
- Vektör Veri Tabanı: FAISS (IndexFlatL2)
- Yaratım (Generator): google/flan-t5-base (transformers pipeline)
- Arayüz: Gradio


## Kurulum & Çalıştırma
1. GitHub'dan klonlayın veya dosyaları indirin.
2. `pip install -r requirements.txt`
3. `python <script>.py` veya `jupyter/colab` üzerinde notebook'u çalıştırın.


## Kullanım
- Colab üzerinde tüm hücreleri çalıştırın.
- Gradio arayüzü açıldığında soru girip test edin.


## Deploy
- İsterseniz bu Gradio uygulamasını HuggingFace Spaces veya başka bir ortamda deploy edebilirsiniz.
'''
with open('README.md', 'w') as f:
  f.write(readme)


print('requirements.txt ve README.md oluşturuldu.')

requirements.txt ve README.md oluşturuldu.
